In [14]:

# personal note, CTC Tensorflow documentation is not at all clear and intuitive , so if confused refer to any example

# trying to implement word level CTC

In [54]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc

In [55]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [56]:
input_mfcc = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [57]:
input_mfcc.shape

(299, 13)

In [ ]:
# we also need a fixed vocabulary 
import re

word_dictionary = {'hello':0 , 'world':1 , ' ':2}


def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
        if word in word_dictionary:
            print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [59]:
word_to_index('hello world hello world')

hello
world
hello
world


[0, 2, 1, 2, 0, 2, 1]

In [6]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])


In [7]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    #print(outputs)
    #temp=tf.convert_to_tensor(outputs)[:,-1,:]
    
   # temp=temp[:,-1,:]
    #return outputs
    return tf.matmul(outputs, weights['out']) + biases['out']
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [9]:
logits = RNN(X, weights, biases)
#prediction = tf.nn.softmax(logits)
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])

loss = ctc_ops.ctc_loss( targets, logits , seq_len)
cost = tf.reduce_mean(loss)


In [ ]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [10]:
init = tf.global_variables_initializer()

In [11]:
input_x = np.random.rand(128,28,28)

In [13]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    
    
    final=np.array(sess.run(logits, feed_dict={X: input_x}))
    print(final.shape)

(128, 28, 10)


hello
world
hello
world


[0, 2, 1, 2, 0, 2, 1]